In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import logging

import cipy

logger = logging.getLogger()
handler = logger.handlers[0]
handler.setLevel(logging.CRITICAL)

conn_creds = cipy.db.get_conn_creds('DATABASE_URL')
pgdb = cipy.db.PostgresDB(conn_creds)

---

## User Management

- create new user accounts and store passwords in a secure format
- delete existing user accounts (along with any owned reviews)
- user login

In [4]:
%run ../scripts/create_user.py --test

Enter user name: Sam C
Enter user email: samc@gmail.com
Confirm user email: samc@gmail.com
Enter password: ········
Confirm password: ········


2016-07-12 22:27:25,784 - create_user - INFO - created user id=3: {'owned_review_ids': None, 'name': 'Sam C', 'email': 'samc@gmail.com', 'created_ts': '2016-07-13T01:48:11.754032Z', 'review_ids': None}


In [5]:
list(pgdb.run_query('SELECT * from users'))

[{'created_ts': datetime.datetime(2016, 7, 7, 17, 56, 7),
  'email': 'burtdewilde@gmail.com',
  'name': 'Burton DeWilde',
  'owned_review_ids': [1],
  'password': '$2a$06$5qfLF4y/sfXkc8XhZ360i.48V5GaQfxF5Uy8zVJcO6dLmUqX9JGie',
  'review_ids': [1],
  'user_id': 1},
 {'created_ts': datetime.datetime(2016, 7, 13, 1, 48, 12),
  'email': 'samc@gmail.com',
  'name': 'Sam C',
  'owned_review_ids': None,
  'password': '$2a$06$MHhP.XjRGfSO7eiDiNLFieJEGoOXxQruc6.Sd3nb.vImvoAuO09x.',
  'review_ids': None,
  'user_id': 3}]

In [9]:
%run ../scripts/login_user.py

Enter email: burtdewilde@gmail.com
Enter password: ········


2016-07-12 21:37:51,681 - login_user - INFO - Welcome, Burton DeWilde id=1


In [2]:
%run ../scripts/delete_user.py --user_id=3 --test

2016-07-12 21:39:29,460 - delete_user - INFO - deleted user id=1 from reviews (TEST)
2016-07-12 21:39:29,460 - delete_user - INFO - deleted user id=1 (TEST)


---

## Review Management

- create new reviews (with user as owner)
- delete existing owned reviews
- invite/uninvite other users to collaborate on existing reviews
- assign other user as owned review's new owner

In [2]:
%run ../scripts/create_review.py --user_id=1 --test

Review name: My Great Systematic Review
Review description (optional):
lorem ipsem dolor sit amen


2016-07-12 21:59:26,941 - create_review - INFO - created review (TEST): {'name': 'My Great Systematic Review', 'owner_user_id': 1, 'description': 'lorem ipsem dolor sit amen', 'created_ts': '2016-07-13T01:48:11.753010Z', 'user_ids': [1]}


In [3]:
%run ../scripts/delete_review.py --user_id=1 --review_id=1 --test

2016-07-12 21:59:45,881 - delete_review - INFO - deleted review id=1 (TEST)


In [37]:
%run ../scripts/manage_collaborators.py --owner_user_id=1 --review_id=1 --add_user_emails "samc@gmail.com"

2016-07-12 22:40:52,400 - manage_collaborators - INFO - user id=3 added as collaborator to review id=1 


In [38]:
print('review:', list(pgdb.run_query('SELECT review_id, user_ids FROM reviews WHERE review_id=1')))
print('user:', list(pgdb.run_query('SELECT user_id, review_ids FROM users WHERE user_id=3')))

review: [{'review_id': 1, 'user_ids': [1, 3]}]
user: [{'user_id': 3, 'review_ids': [1]}]


In [39]:
%run ../scripts/manage_collaborators.py --owner_user_id=1 --review_id=1 --remove_user_emails "samc@gmail.com"

2016-07-12 22:40:55,110 - manage_collaborators - INFO - user id=3 removed as collaborator to review id=1 


In [40]:
print('review:', list(pgdb.run_query('SELECT review_id, user_ids FROM reviews WHERE review_id=1')))
print('user:', list(pgdb.run_query('SELECT user_id, review_ids FROM users WHERE user_id=3')))

review: [{'review_id': 1, 'user_ids': [1]}]
user: [{'user_id': 3, 'review_ids': []}]


---

## Review Planning

- facilitate systematic review planning while also gathering structured data that informs and is informed by the citation pre-screening process; user entry of the following fields:
    - objective
    - research questions, ranked
    - PICO statements
    - grouped keyterms (with automatic boolean search query generation)
    - data sources
    - inclusion/exclusion criteria, with shorthand labels

In [8]:
%run ../scripts/plan_review.py --user_id=1 --review_id=1 --test


PROJECT PLAN

Objective:
To assess and characterize the current state and distribution of the existing evidence base around the causal linkages between both positive and negative effects of nature conservation and human well-being.

Research Questions:
 0 What are the impacts of nature conservation interventions on different domains of human well-being in developing countries?
 1 What is the current state and distribution of evidence?
 2 What types of impacts from conservation interventions on human well-being are measured?
 3 What types of ecosystem services are explicitly associated with the impacts of conservation interventions on human well-being?
 4 What populations are affected by conservation and/ or focus of studies?
 5 How does the evidence base align with major priorities and investments of implementing agencies?

PICO:
- Population    : Human populations, including individuals, households, communities or nation states in non-OECD countries
- Intervention  : Adoption or impl

2016-07-20 23:51:50,892 - plan_review - INFO - valid record: review_id=1 with {'objective', 'research_questions', 'selection_criteria', 'keyterms', 'pico'}, 



PROJECT PLAN

Objective:
To assess and characterize the current state and distribution of the existing evidence base around the causal linkages between both positive and negative effects of nature conservation and human well-being.

Research Questions:
 0 What are the impacts of nature conservation interventions on different domains of human well-being in developing countries?
 1 What is the current state and distribution of evidence?
 2 What types of impacts from conservation interventions on human well-being are measured?
 3 What types of ecosystem services are explicitly associated with the impacts of conservation interventions on human well-being?
 4 What populations are affected by conservation and/ or focus of studies?
 5 How does the evidence base align with major priorities and investments of implementing agencies?

PICO:
- Population    : Human populations, including individuals, households, communities or nation states in non-OECD countries
- Intervention  : Adoption or impl

---

## Citation Ingestion and De-duplication

- load citations from RIS or BibTex files then parse, standardize, sanitize, validate, and store the data
- identify duplicate citations using a sophisticated model and assign the most complete record in a set of duplicates as the "canonical" record

In [13]:
%run ../scripts/ingest_citations.py --citations ../data/raw/citation_files/phase_2_demo_citations.ris --user_id=1 --review_id=1 --test

2016-07-11 22:24:20,813 - ingest_citations - INFO - parsing records in ../data/raw/citation_files/phase_2_demo_citations.ris
2016-07-11 22:24:20,814 - ingest_citations - INFO - valid record: Ecological protection and well-being, 2013
2016-07-11 22:24:20,815 - ingest_citations - INFO - valid record: The economic value of forest ecosystems, 2001
2016-07-11 22:24:20,816 - ingest_citations - INFO - valid record: Contribution of tourism development to protected area management: Local stakeholder perspectives, 2009
2016-07-11 22:24:20,817 - ingest_citations - INFO - 3 valid records inserted into appname db (TEST)


In [17]:
num_citations = list(pgdb.run_query('SELECT COUNT(1) FROM citations WHERE review_id = 1'))[0]['count']
print('total # citations =', num_citations)

total # citations = 28709


In [7]:
%run ../scripts/dedupe_records.py --review_id=1 --threshold=auto --settings=../models/dedupe_citations_settings --test

2016-07-11 22:19:58,881 - dedupe_records - INFO - reading dedupe settings from ../models/dedupe_citations_settings
2016-07-11 22:20:19,236 - dedupe_records - INFO - duplicate threshold = 0.827943
2016-07-11 22:20:19,879 - dedupe_records - INFO - found 361 duplicate clusters
2016-07-11 22:20:20,020 - dedupe_records - INFO - inserted 726 records into duplicates db (TEST)


In [2]:
query = """
SELECT canonical_citation_id, array_agg(citation_id) AS citation_ids, AVG(duplicate_score) AS avg_score
FROM duplicates
GROUP BY 1 HAVING AVG(duplicate_score) < 0.9 ORDER BY 1 ASC
LIMIT 1
"""
dupes = list(pgdb.run_query(query))[0]
print('citations {} are duplicates with avg. duplicate score = {}'.format(
        dupes['citation_ids'], round(dupes['avg_score'], 6)))

query = """
SELECT citation_id, authors, title, abstract, publication_year, doi
FROM citations
WHERE citation_id = ANY(%(citation_ids)s)
"""
for record in pgdb.run_query(query, {'citation_ids': dupes['citation_ids']}):
    cipy.db.db_utils.present_citation(record)

citations [497, 496] are duplicates with avg. duplicate score = 0.827182

TITLE:    INTEGRATIVE SOCIAL WORK APPROACH AS A CONTEXT FOR UNDERSTANDING THE INDIVIDUAL SOCIAL CARE PLAN
YEAR:     2009
AUTHORS:  Ajdukovic, M; Urbanc, K
ABSTRACT: The article deals with the issue of introducing the individual social core plan as one of the initiatives pertaining to the long-awaited social core system reform. The ideas of the individual social care plan are placed within a theoretical framework of the integrative social work approach and in the context of numerous changes which have occurred in the lost twenty years at the conceptual and practical level of offering integrated and coordinated services to service users, ie case management, care management, person-centred planning, etc. Based on the experience that, in order to change the organisation of the Centres for Social Care and develop efficient social care services, the changes to the legal framework are not sufficient but it is necessary 

---

## Initial Ranking of Citations

- sample citations ranked by overlap with keyterms; user pre-screens citations until 10 have been included and 10 have been excluded
- based on included/excluded citations, rank citations by ratio of relevant to irrelevant keyterms and present those most likely to be relevant to the user for pre-screening

---

## Refinement of Search Keyterms

- based on included/excluded citations, create lists of strongly relevant and irrelevant keyterms that can be used to refine initial set of keyterms